# Accuracy 

## Overflow and underflow

In python 3, integers can hold arbitrarily large values (within limits of system memory).

In [ ]:
bigNum = 2**1024

bigNum

However, `float`s can only represent values between $\sim\!10^{-308}$ (there are subtle caveats to this statement) and $\sim\!10^{308}$.

In [ ]:
1e308, 1e308*10

The behavior above is referred to as "overflow."  `inf` is short for infinity, and behaves roughly as one would expect.

In [ ]:
inf=float("inf")

print(type(inf))
print(inf/1e308,
      inf/1e309)

The opposite scenario, in which a calculated value is too small, is referred to as "underflow."

In [ ]:
1e-323, 1e-323/10

The case of underflow is more subtle than overflow.  There is a gradual loss of precision for values below $\sim\!10^{308}$.

In [ ]:
smallNum = 1.123456789123456789e-307  

while smallNum>0:
    print(smallNum)
    smallNum /= 10

Other numerical types have limitations as well.

## `sys` module

Python has a `sys` module that allows you to check parameters of your computer: https://docs.python.org/3.6/library/sys.html

In [ ]:
import sys
sys.platform

The attribute `float_info` gives information on the maximum and minimum `float` values, as well as other info.

In [ ]:
sys.float_info

## Warning

Logical errors can happen when you are dealing with overflow and underflow situations.


In [ ]:
a, b = 1.0e500, 7.0e500

a, b, a == b

Python uses `nan` (not a number) to represent undefined or unpresentable values. 

In [ ]:
c = a/b

c

`nan` has the unusual property that it is not equal to itself.

In [ ]:
c == c

The math module includes functions to test if a variable is `inf` or `nan`.

In [ ]:
import math

math.isinf(a), math.isnan(c)

## Float precision and numerical error

Since we have finite bits to represent variables, irrational numbers must be rounded.  However, rational numbers with more than 16 significant figures will also be rounded, leading to rounding errors.

In [ ]:
0.1 + 0.1 + 0.1

Why didn't python calculate 0.3?  To understand some issues related to rounding, we must consider the binary representation of variables.  In memory, numerical values are represented in base 2 (binary).  While 0.1 is a rational number in base 10, in base 2 it is the irrational number 0.0<span style="color:red">0011</span><span style="color:orange">0011</span><span style="color:yellow">0011</span><span style="color:green">0011</span><span style="color:blue">0011</span><span style="color:purple">0011</span>...  Therefore, this value can't be accurately represented with finite bits, resulting in rounding error.

------------------

### Equality testing

Due to issues related to numerical precision, you should never test for equality of two `float`s.

In [ ]:
x=49.0

x*(1/49.0) == 1

Instead, you should test if two `float`s have a difference less than some small number.

In [ ]:
x = 1.1+2.2
epsilon = 1e-12

print(x == 3.3, abs(x-3.3)<epsilon)

Alternatively, you can use the `isclose` function (from the math module).

In [ ]:
import math

math.isclose(x, 3.3)

NumPy also provides an `allclose` function for element-by-element equality testing of `array`s.

In [ ]:
import numpy as np

x = np.array([[3.141592, -0.1], 
              [-0.1,     0.1]])
y = np.array([[math.pi, -0.1], 
              [-0.1,     0.1]])

np.allclose(x,y)

Also note that floating point addition is not associative.

In [ ]:
a, b, c = 1e14, 25.44, 0.74

(a+b)+c == a+(b+c)

Nor is it distributive.

In [ ]:
a, b, c, = 100, 0.1, 0.2

a*(b+c) == a*b + a*c

## Loss of signficance

Suppose we want to calculate the difference betwen two numbers, 1.2345432 and 1.23451.  With full precision, the difference is 0.00000332.  

However, if we calculated the difference on a machine with only 6 digits of precision, (after rounding) we would calculate the difference as $1.23454-1.23451 = 0.00003$.  Initially, we had two numbers, with 8 and 6 significant figures.  However, the result of our calculation has only a single significant figure.  This is a common phenomenon when taking the difference of two similar numbers.

A similar phenomenon occurs when a small number is added/subtracted to a large number.  Suppose we want to add the numbers 12345.6 and 0.123456.  Again, on a machine with only 6 digits of precision, we would calculate the sum as $12345.6 + 0.123456 = 12345.7$, resulting in an error of 0.023456.

In reality, python has 15 digits of precision, so this is a smaller effect.  But it can still cause problems.  

Let's assume we want to calculate the difference between two numbers, $1$ and $1+10^{-14}\sqrt{2}$.  With infinite precision, the difference is $10^{-14}\sqrt{2}$ or approximately $1.414213562\times10^{-14}$.  However, python gives a different result.

In [ ]:
x = 1
y = 1 + math.sqrt(2)*1e-14

y-x

This is accurate only to 1 decimal place.

To avoid such issues, the `fsum()` function (within the math module) can be used: https://docs.python.org/3/library/math.html#math.fsum

In [23]:
l = [.1, .1, .1, .1, .1, .1, .1, .1, .1, .1]

sum(l), math.fsum(l)

(0.9999999999999999, 1.0)